# Creating and Valuing a CDS Index

Example CDS Index Valuation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.products.credit import *

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 19:38 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



In [2]:
tradeDate = FinDate(7, 2, 2006)
settlementDate = tradeDate.addDays(1)
print(settlementDate)

WED 08 FEB 2006


## Build Libor Curve

Build the Libor curve to value cashflows

In [3]:
depos = []
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinLiborSwapTypes.PAYER
swap1 = FinLiborSwap(settlementDate,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = FinLiborSwap(settlementDate,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = FinLiborSwap(settlementDate,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = FinLiborSwap(settlementDate,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = FinLiborSwap(settlementDate,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

## Build Index Curve

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

In [4]:
stepInDate = tradeDate.addWorkDays(1)
valuationDate = stepInDate

In [5]:
indexCoupon = 0.0048375
maturityDate = FinDate(20, 6, 2010)

cdsMarketContracts = []
cds = FinCDS(valuationDate, maturityDate, indexCoupon)
cdsMarketContracts.append(cds)

recoveryRate = 0.40
indexCurve = FinCDSCurve(valuationDate,cdsMarketContracts,liborCurve,recoveryRate)

## Create CDS Index and Perform Valuation

The CDS Index is treated as a CDS with the coupon equal to the CDS Index coupon

In [6]:
cdsRecovery = 0.40
notional = 10.0 * ONE_MILLION
longProtection = True
indexCoupon = 0.004 

cdsIndexContract = FinCDS(stepInDate, maturityDate, indexCoupon, notional, longProtection)

In [7]:
spd = cdsIndexContract.parSpread(valuationDate, indexCurve, cdsRecovery) * 10000
print("PAR SPREAD", spd)

PAR SPREAD 48.37497296446841


In [8]:
v = cdsIndexContract.value(valuationDate, indexCurve, cdsRecovery)
print("FULL VALUE", v['full_pv'])
print("CLEAN VALUE", v['clean_pv'])

FULL VALUE 27019.338807193562
CLEAN VALUE 32574.894362749095


In [9]:
p = cdsIndexContract.cleanPrice(valuationDate, indexCurve, cdsRecovery)
print("CLEAN PRICE", p)

CLEAN PRICE 99.67425109514956


In [15]:
accruedDays = cdsIndexContract.accruedDays()
print("ACCRUED DAYS", accruedDays)

accruedInterest = cdsIndexContract.accruedInterest()
print("ACCRUED COUPON", accruedInterest)

ACCRUED DAYS 50
ACCRUED COUPON -5555.555555555556


In [16]:
protPV = cdsIndexContract.protectionLegPV(valuationDate, indexCurve, cdsRecovery)
print("PROTECTION LEG PV", protPV)

PROTECTION LEG PV 188156.98161737667


In [17]:
premPV = cdsIndexContract.premiumLegPV(valuationDate, indexCurve, cdsRecovery)
print("PREMIUM LEG PV", premPV)

PREMIUM LEG PV 161137.6428101153


In [18]:
rpv01 = cdsIndexContract.riskyPV01(valuationDate, indexCurve)
print("FULL  RPV01", rpv01['full_rpv01'])
print("CLEAN RPV01", rpv01['clean_rpv01'])

FULL  RPV01 4.028441070252883
CLEAN RPV01 3.8895521813639937


Copyright (c) 2020 Dominic O'Kane